# Import necessary modules

In [ ]:
import pathlib

import ipywidgets as widgets

import ipyniivue
from ipyniivue import NiiVue, SliceType, download_dataset

# Download required data

In [ ]:
DATA_FOLDER = pathlib.Path(ipyniivue.__file__).parent / "images"

download_dataset(
    "https://niivue.com/demos/images/",
    dest_folder=DATA_FOLDER,
    files=[
        "mni152.nii.gz",
        "dpsv.trx",
    ],
)

# Create NiiVue instance

In [ ]:
nv = NiiVue(show_3d_crosshair=True, back_color=(0.8, 0.8, 1, 1))

nv.opts.is_colorbar = True
nv.set_slice_type(SliceType.RENDER)
nv.set_clip_plane(-0.1, 270, 0);

# Load the volume and mesh data

In [ ]:
nv.load_volumes(
    [
        {"path": DATA_FOLDER / "mni152.nii.gz"},
    ]
)

nv.load_meshes(
    [
        {"path": DATA_FOLDER / "dpsv.trx", "rgba255": [0, 142, 0, 255]},
    ]
)

nv.meshes[0].colormap = "blue"
nv.meshes[0].rgba255 = (0, 255, 255, 255)

# Create sliders and dropdowns

In [ ]:
radius_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    description="Radius",
)

length_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=80,
    description="Length",
)

dither_slider = widgets.IntSlider(
    value=1,
    min=0,
    max=10,
    description="Dither",
)

fiber_coloration_options = [
    ("Global direction", "Global"),
    ("Local direction", "Local"),
    ("Fixed", "Fixed"),
    ("First Per Vertex Type (if available)", "DPV0"),
    ("First Per Streamline Type (if available)", "DPS0"),
]
fiber_color_dropdown = widgets.Dropdown(
    options=fiber_coloration_options,
    value="Global",
    description="Fiber Coloration:",
)

fiber_reduction_options = [("100%", 1), ("50%", 2), ("25%", 4), ("10%", 10)]
fiber_reduction_dropdown = widgets.Dropdown(
    options=fiber_reduction_options,
    value=1,
    description="Fiber Reduction",
)

# Set up callbacks

In [ ]:
def on_radius_change(change):
    """Set fiber radius."""
    nv.set_mesh_property(nv.meshes[0].id, "fiber_radius", change["new"] * 0.1)


radius_slider.observe(on_radius_change, names="value")


def on_length_change(change):
    """Set fiber length."""
    nv.set_mesh_property(nv.meshes[0].id, "fiber_length", change["new"])


length_slider.observe(on_length_change, names="value")


def on_dither_change(change):
    """Set fiber dither."""
    nv.set_mesh_property(nv.meshes[0].id, "fiber_dither", change["new"] * 0.1)


dither_slider.observe(on_dither_change, names="value")


def on_fiber_color_change(change):
    """Set fiber color."""
    nv.set_mesh_property(nv.meshes[0].id, "fiber_color", change["new"])


fiber_color_dropdown.observe(on_fiber_color_change, names="value")


def on_fiber_reduction_change(change):
    """Set fiber decimation stride."""
    nv.set_mesh_property(nv.meshes[0].id, "fiber_decimation_stride", change["new"])


fiber_reduction_dropdown.observe(on_fiber_reduction_change, names="value")

# Display all

In [ ]:
controls = widgets.VBox(
    [
        radius_slider,
        length_slider,
        dither_slider,
        fiber_color_dropdown,
        fiber_reduction_dropdown,
    ]
)

widgets.VBox(
    [
        controls,
        nv,
    ]
)